### Computational Guided Inquiry for Modeling Earth's Climate (Neshyba, 2024)

# Cambio 4.0

## The terrestrial sequestration feedback
In previous versions of Cambio, the atmosphere-to-land carbon flux was calculated as $F_{atm->land} = k_{al0} +  k_{al1} \times [C_{atm}]$, in which $k_{al1}$ was described as a measure of the strength of Earth's natural $CO_2$ fertilization capability. Because this capability is a free ecosystem service provided by (mostly) forest photosynthesis, part of Earth system stewardship would be to help ensure that $k_{al1}$ stays high. 

There is evidence, however, that $k_{al1}$ is getting smaller with increasing global temperature. Since that reduction itself drives increasing temperatures, we can understand it as a feedback mechanism -- which we'll call the "terrestrial sequestration feedback" (or TSF) effect. In Cambio 4.0, we'll represent this reduction algorithmically by multiplying $k_{al1}$ by a sigmoid function whose value drops from a pre-industrial value of 1, to a smaller value in a warmer world. Thus, the equations of Cambio 4.0 are written

$$
F_{land->atm} =  k_{la} \ \ \ (1) 
$$

$$
F_{atm->land} = k_{al0} +  k_{al1} \times \sigma_{floor}(T_{anomaly}) \times [C_{atm}] \ \ \ (2)
$$

$$
F_{ocean->atm} = k_{oa} \times (1+DC\times T_{anomaly}) [C_{ocean}] \ \ \ (3)
$$

$$
F_{atm->ocean} = k_{ao} [C_{atm}] \ \ \ (4)
$$

$$
F_{human->atm} = \epsilon(t) \ \ \ (5)
$$

The chief challenge in this approach consists of finding realistic values of parameters that control $\sigma_{floor}(T_{anomaly})$. As you will probably recall, there are (generically) three of these:
- $T_{anomaly}^*$, the tipping point temperature at which $\sigma_{floor}(T_{anomaly})$ transitions from its pre-industrial to its post-warming value.
- $\Delta T$, a measure of the abruptness of that transition
- $\sigma_{floor}$, the post-warming (smaller) value of $\sigma$

Recently, Ke et al, 2024 (https://arxiv.org/abs/2407.12447) have documented declining rates of atmosphere-to-land carbon flux, including an abrupt reduction in the year 2023. Geographically, it appears that reduction in the rate of sequestration in the Amazon basin is chiefly responsible. More to our present purpose, the paper describes these declining rates in a way that enables us to assign realistic values to $T_{anomaly}^*$, $\Delta T$, and $\sigma_{floor}$. That means we have everything we need to move forward with adding terrestrial sequestration feedback to our model. 

To summarize:

- Cambio 1.0 has the five basic equations of motion, with no feedbacks
- Cambio 2.0 adds in Henry's Law feedbacks
- Cambio 3.0 adds in ice-albedo feedback
- Cambio 4.0 adds in terrestrial sequestion feedback

## The idea of this CGI
Here, the idea is to see how addition of the terrestrial sequestration feedback mechanism affects the magnitude and timing of peak warming. To facilitate this, the process of running our Cambio simulations has been greatly streamlined. Although doing so comes at the cost of burying the algorithms inside Python libraries, it makes everything a lot easier to use.

## Uploading your climate emissions scenario
As before, you'll need to upload a climate emissions scenario file to the Cambio4.0 folder. Make this one you generated by ScheduledFlowsWithLTE.

## Learning goals
1. I can describe the terrestrial sequestration feedback (TSF) mechanism, and the parameters ($T_{anomaly}^*$, $\Delta T$, and $\sigma_{floor}$) that parameterize it.
1. I can explain how it happens that while the impact of feedbacks such as the TSF can be estimated from the parameters, only an integrated simulation can provide its full impact.
1. I can describe how TSF affects key climate predictions, including the magnitude and timing of anthropogenic maximum warming.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys; sys.path.append('/home'); import MECLib as CL

In [ ]:
# %matplotlib inline
# plt.rcParams["figure.figsize"] = (12, 8)
# plt.rcParams['font.size'] = 18
%matplotlib notebook

### Loading your favorite scheduled flow
In the cell below, load in your scheduled flows file. It'll be most convenient if you use the following naming convention: 

    epstime, eps, epsdictionary = CL.GetMyScenario('...')

(but of course supplying the name of your own scheduled flows file). 

You should display the dictionary too:

    display(epsdictionary)

In [ ]:
# your code here 


### Creating a dictionary for climate parameters
In the cell below, we use the CreateClimateParams function to create a dictionary of climate parameters.

In [ ]:
ClimateParams = CL.CreateClimateParams(epsdictionary)
display(ClimateParams)

### Getting a sense of the impact of the terrestrial sequestration feedback
The cell below provides a sense of the potential impact of the TSF (terrestrial sequestration feedback), by plotting $F_{atm->land} = k_{al0} +  k_{al1} \times \sigma_{floor}(T_{anomaly}) \times [C_{atm}]$ as a function of the temperature anomaly we expect that will drive the TSF. Parameters controlling the sigmoid function ($T_{anomaly}^*$, $\Delta T$, and $\sigma_{floor}$) are based on a fit to data given in Ke et al, 2024 (https://arxiv.org/abs/2407.12447).

We should emphasize that graphs like the one shown below provide only a *prediction of the consequence of a climate driver* (i.e., the reduction in terrestrial sequestration as a function of warming). Assessment of the full impact of this driver can only be made from an integrated simulation, in which the driver is included as part of the solution (Euler loop) of the Eqs. 1-5 in the Introduction. 

In [ ]:
# A range of atmospheric CO2 amounts (preindustrial to 2x that)
C_atm_array = np.linspace(615,2*615)
C_atm_array_ppm = C_atm_array/2.12

# Get the temperature anomaly that would result from that, assuming no ice-albedo feedback
T_anomaly = CL.Diagnose_T_anomaly(C_atm_array, 0.3, ClimateParams)

# Other parameters we'll need for this comparison 
k_al0 = ClimateParams['k_al0']
k_al1 = ClimateParams['k_al1']
k_al1_Tstar = ClimateParams['k_al1_Tstar']
k_al1_deltaT = ClimateParams['k_al1_deltaT']
fractional_k_al1_floor = ClimateParams['fractional_k_al1_floor']

# Terrestrial atmosphere-to-land flux with and without terrestrial sequestration feedback
F_al_without_tsf = k_al0 + k_al1*C_atm_array
F_al_with_tsf_extrap = k_al0 + k_al1*C_atm_array*fractional_k_al1_floor
F_al_with_tsf = k_al0 + k_al1*C_atm_array*CL.sigmafloor(T_anomaly,k_al1_Tstar,k_al1_deltaT,fractional_k_al1_floor)

# Plot them side by side
iextrap = int(len(C_atm_array_ppm)/2)
plt.figure()
plt.plot(C_atm_array_ppm,F_al_without_tsf,'k',label="No TSF")
plt.plot(C_atm_array_ppm,F_al_with_tsf,'r',label="With TSF (tipping point $T^*_{anomaly}$="+str(k_al1_Tstar)+"$^oC$)")
plt.plot(C_atm_array_ppm[iextrap:],F_al_with_tsf_extrap[iextrap:],'r.')
plt.xlabel('Carbon in atmosphere (ppm)')
plt.ylabel('F_al (GtC/yr)')
plt.title('Estimated impact of terrestrial sequestration feedback (TSF) on atm-to-land flux')
plt.legend()
plt.grid()

### Pause for analysis
1. Given the current concentration of $CO_2$ in the atmosphere, how much has $CO_2$ fertilization already been impaired so far? 
1. How much impairment can we expect if atmospheric $CO_2$ reaches double its pre-industrial amount ($290 \times 2 = 580 \ ppm$)? Provide your answers in GtC/yr.

YOUR ANSWER HERE

### Running the various Cambio versions
The cell below runs Cambio versions 1-4.

In [ ]:
ClimateState_Cambio1_list = CL.run_Cambio(CL.PropagateClimateState_Cambio1, ClimateParams, epstime, eps)
ClimateState_Cambio2_list = CL.run_Cambio(CL.PropagateClimateState_Cambio2, ClimateParams, epstime, eps)
ClimateState_Cambio3_list = CL.run_Cambio(CL.PropagateClimateState_Cambio3, ClimateParams, epstime, eps)
ClimateState_Cambio4_list = CL.run_Cambio(CL.PropagateClimateState_Cambio4, ClimateParams, epstime, eps)

### Here we specify what we want to plot using a Python list, and then plot the results for each model 
Each element of this list is either itself a list of items (the first two) or a simple item (last two).

In [ ]:
items_to_plot = [['C_atm','C_ocean'],['F_ocean_net','F_land_net','F_ha'],'T_anomaly','OceanSurfacepH', 'albedo']
CL.Climatestate_list_plots(ClimateState_Cambio1_list,items_to_plot,'Cambio1')
CL.Climatestate_list_plots(ClimateState_Cambio2_list,items_to_plot,'Cambio2')
CL.Climatestate_list_plots(ClimateState_Cambio3_list,items_to_plot,'Cambio3')
CL.Climatestate_list_plots(ClimateState_Cambio4_list,items_to_plot,'Cambio4')

### Pause for analysis
After studying the graphs above, provide in the cell below answers to the following questions. It will be helpful to refer to the max/min printouts as you do so.

1. How much greater is the peak temperature that we can attribute to all feedbacks, compared to none? (i.e., Cambio 4.0 compared to Cambio 1.0)
1. How does the timing of peak temperature increase due to feedbacks? (again, comparing timing of peak temperatures predicted by Cambio 4.0 to Cambio 1.0).
1. How much peak warming can we attribute to terrestrial sequestration feedback alone? (i.e., Cambio 4.0 compared to Cambio 3.0)

YOUR ANSWER HERE

### Refresh/save/validate
Double-check everything is OK, and press the "Validate" button (as usual).

### Close/submit/logout
Close, submit, and log out.